In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls /content/drive/MyDrive/dataset

add_stim.mat		combined_control_data.mat  d2.mat  d7.mat		splitset
adhderps		controlerps		   d3.mat  erp_features.csv	sub_name_stim.mat
adhd_stim.mat		controlset		   d4.mat  erp_features.gsheet	y_stim.mat
chan.mat		control_stim.mat	   d5.mat  plots
combined_adhd_data.mat	d1.mat			   d6.mat  readme.txt


In [3]:
pip install mne

In [4]:
import os
import mne
import scipy.io as sio
import pandas as pd

# Load selected channel indices
def get_selected_channel_indices(chan_mat_path):
    chan_info = sio.loadmat(chan_mat_path)
    chan_struct = chan_info['chan'][0]
    all_labels = [str(chan['labels'][0]) for chan in chan_struct]

    target_channels = ['FC1', 'Fz', 'C3', 'FC5', 'P3', 'Fp1', 'P7', 'O1',
                       'C4', 'Pz', 'AF4', 'P4', 'Fp2', 'P8', 'O2']

    selected_indices = [i for i, label in enumerate(all_labels) if label in target_channels]
    selected_labels = [all_labels[i] for i in selected_indices]
    return selected_indices, selected_labels

# Print peak info
def print_peak_measures(component, ch, tmin, tmax, lat, amp):
    print(f"--- {component} Component ---")
    print(f"Channel: {ch}")
    print(f"Time Window: {tmin * 1e3:.1f} - {tmax * 1e3:.1f} ms")
    print(f"Peak Latency: {lat * 1e3:.2f} ms")
    print(f"Peak Amplitude: {amp * 1e6:.2f} µV\n")

# Extract ERP component
def extract_erp_features(evoked, component_name, tmin, tmax, mode):
    ch, lat, amp = evoked.get_peak(
        ch_type="eeg", tmin=tmin, tmax=tmax, mode=mode,
        return_amplitude=True
    )
    print_peak_measures(component_name, ch, tmin, tmax, lat, amp)
    return ch, lat * 1e3, amp * 1e6  # Convert latency to ms and amplitude to µV

# Main processing with CSV export
def process_adhd_erp(erp_folder, chan_mat_path, output_csv_path):
    _, selected_labels = get_selected_channel_indices(chan_mat_path)
    rows = []

    for fname in os.listdir(erp_folder):
        if fname.endswith("_erp.fif"):
            print(f"\nProcessing: {fname}")
            evoked = mne.read_evokeds(os.path.join(erp_folder, fname), condition=0)
            evoked.pick_channels(selected_labels)

            # Extract features
            ch_p1, lat_p1, amp_p1 = extract_erp_features(evoked, "P1", 0.08, 0.12, mode="pos")
            ch_n1, lat_n1, amp_n1 = extract_erp_features(evoked, "N1", 0.12, 0.18, mode="neg")
            ch_p3, lat_p3, amp_p3 = extract_erp_features(evoked, "P3", 0.30, 0.50, mode="pos")

            # Add to result list
            rows.append({
                "Filename": fname,
                "Channel_P1": ch_p1, "Latency_P1 (ms)": lat_p1, "Amplitude_P1 (µV)": amp_p1,
                "Channel_N1": ch_n1, "Latency_N1 (ms)": lat_n1, "Amplitude_N1 (µV)": amp_n1,
                "Channel_P3": ch_p3, "Latency_P3 (ms)": lat_p3, "Amplitude_P3 (µV)": amp_p3,
            })

    # Create DataFrame and save to CSV
    df = pd.DataFrame(rows)
    df.to_csv(output_csv_path, index=False)
    print(f"\n ERP features saved to {output_csv_path}")

In [5]:
erp_folder = '/content/drive/MyDrive/dataset/adhderps'
chan_mat_path = '/content/drive/MyDrive/dataset/chan.mat'
output_csv_path = '/content/drive/MyDrive/dataset/erp_features.csv'

process_adhd_erp(erp_folder, chan_mat_path, output_csv_path)


Processing: 14_train_erp.fif
Reading /content/drive/MyDrive/dataset/adhderps/14_train_erp.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    1500.00 ms (1)
        0 CTF compensation matrices available
        nave = 238 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
No baseline correction applied
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
--- P1 Component ---
Channel: P8
Time Window: 80.0 - 120.0 ms
Peak Latency: 117.19 ms
Peak Amplitude: 26977302.55 µV

--- N1 Component ---
Channel: O1
Time Window: 120.0 - 180.0 ms
Peak Latency: 179.69 ms
Peak Amplitude: -53820949.55 µV

--- P3 Component ---
Channel: P7
Time Window: 300.0 - 500.0 ms
Peak Latency: 339.84 ms
Peak Amplitude: 47824584.96 µV


Processing: 18_train_erp.fif
Reading /content/drive/MyDrive/dataset/adhderps/18_train_erp.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.0

<ipython-input-4-ce1d908fed66>:44: RuntimeWarning: This filename (/content/drive/MyDrive/dataset/adhderps/14_train_erp.fif) does not conform to MNE naming conventions. All evoked files should end with -ave.fif, -ave.fif.gz, _ave.fif or _ave.fif.gz
  evoked = mne.read_evokeds(os.path.join(erp_folder, fname), condition=0)
<ipython-input-4-ce1d908fed66>:44: RuntimeWarning: This filename (/content/drive/MyDrive/dataset/adhderps/18_train_erp.fif) does not conform to MNE naming conventions. All evoked files should end with -ave.fif, -ave.fif.gz, _ave.fif or _ave.fif.gz
  evoked = mne.read_evokeds(os.path.join(erp_folder, fname), condition=0)
<ipython-input-4-ce1d908fed66>:44: RuntimeWarning: This filename (/content/drive/MyDrive/dataset/adhderps/46_train_erp.fif) does not conform to MNE naming conventions. All evoked files should end with -ave.fif, -ave.fif.gz, _ave.fif or _ave.fif.gz
  evoked = mne.read_evokeds(os.path.join(erp_folder, fname), condition=0)
<ipython-input-4-ce1d908fed66>:44:

Isotrak not found
    Found the data of interest:
        t =       0.00 ...    1500.00 ms (1)
        0 CTF compensation matrices available
        nave = 125 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
No baseline correction applied
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
--- P1 Component ---
Channel: P7
Time Window: 80.0 - 120.0 ms
Peak Latency: 117.19 ms
Peak Amplitude: 42312496.19 µV

--- N1 Component ---
Channel: Pz
Time Window: 120.0 - 180.0 ms
Peak Latency: 179.69 ms
Peak Amplitude: -30277614.59 µV

--- P3 Component ---
Channel: P7
Time Window: 300.0 - 500.0 ms
Peak Latency: 410.16 ms
Peak Amplitude: 41102291.11 µV


Processing: 10_train_erp.fif
Reading /content/drive/MyDrive/dataset/adhderps/10_train_erp.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    1500.00 ms (1)
        0 CTF compensation matrices available
        nave = 265 - aspect t

<ipython-input-4-ce1d908fed66>:44: RuntimeWarning: This filename (/content/drive/MyDrive/dataset/adhderps/10_train_erp.fif) does not conform to MNE naming conventions. All evoked files should end with -ave.fif, -ave.fif.gz, _ave.fif or _ave.fif.gz
  evoked = mne.read_evokeds(os.path.join(erp_folder, fname), condition=0)
<ipython-input-4-ce1d908fed66>:44: RuntimeWarning: This filename (/content/drive/MyDrive/dataset/adhderps/17_train_erp.fif) does not conform to MNE naming conventions. All evoked files should end with -ave.fif, -ave.fif.gz, _ave.fif or _ave.fif.gz
  evoked = mne.read_evokeds(os.path.join(erp_folder, fname), condition=0)
<ipython-input-4-ce1d908fed66>:44: RuntimeWarning: This filename (/content/drive/MyDrive/dataset/adhderps/30_train_erp.fif) does not conform to MNE naming conventions. All evoked files should end with -ave.fif, -ave.fif.gz, _ave.fif or _ave.fif.gz
  evoked = mne.read_evokeds(os.path.join(erp_folder, fname), condition=0)
<ipython-input-4-ce1d908fed66>:44:

Isotrak not found
    Found the data of interest:
        t =       0.00 ...    1500.00 ms (1)
        0 CTF compensation matrices available
        nave = 229 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
No baseline correction applied
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
--- P1 Component ---
Channel: P7
Time Window: 80.0 - 120.0 ms
Peak Latency: 117.19 ms
Peak Amplitude: 32564212.80 µV

--- N1 Component ---
Channel: O1
Time Window: 120.0 - 180.0 ms
Peak Latency: 179.69 ms
Peak Amplitude: -26755830.76 µV

--- P3 Component ---
Channel: P7
Time Window: 300.0 - 500.0 ms
Peak Latency: 332.03 ms
Peak Amplitude: 119953048.71 µV


Processing: 31_train_erp.fif
Reading /content/drive/MyDrive/dataset/adhderps/31_train_erp.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    1500.00 ms (1)
        0 CTF compensation matrices available
        nave = 287 - aspect 

...

<ipython-input-4-ce1d908fed66>:44: RuntimeWarning: This filename (/content/drive/MyDrive/dataset/adhderps/42_train_erp.fif) does not conform to MNE naming conventions. All evoked files should end with -ave.fif, -ave.fif.gz, _ave.fif or _ave.fif.gz
  evoked = mne.read_evokeds(os.path.join(erp_folder, fname), condition=0)
<ipython-input-4-ce1d908fed66>:44: RuntimeWarning: This filename (/content/drive/MyDrive/dataset/adhderps/11_train_erp.fif) does not conform to MNE naming conventions. All evoked files should end with -ave.fif, -ave.fif.gz, _ave.fif or _ave.fif.gz
  evoked = mne.read_evokeds(os.path.join(erp_folder, fname), condition=0)
<ipython-input-4-ce1d908fed66>:44: RuntimeWarning: This filename (/content/drive/MyDrive/dataset/adhderps/23_train_erp.fif) does not conform to MNE naming conventions. All evoked files should end with -ave.fif, -ave.fif.gz, _ave.fif or _ave.fif.gz
  evoked = mne.read_evokeds(os.path.join(erp_folder, fname), condition=0)
<ipython-input-4-ce1d908fed66>:44:

Isotrak not found
    Found the data of interest:
        t =       0.00 ...    1500.00 ms (1)
        0 CTF compensation matrices available
        nave = 283 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
No baseline correction applied
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
--- P1 Component ---
Channel: O1
Time Window: 80.0 - 120.0 ms
Peak Latency: 117.19 ms
Peak Amplitude: 55252273.56 µV

--- N1 Component ---
Channel: Fp2
Time Window: 120.0 - 180.0 ms
Peak Latency: 128.91 ms
Peak Amplitude: -20101715.09 µV

--- P3 Component ---
Channel: P7
Time Window: 300.0 - 500.0 ms
Peak Latency: 308.59 ms
Peak Amplitude: 66957977.29 µV


Processing: 21_train_erp.fif
Reading /content/drive/MyDrive/dataset/adhderps/21_train_erp.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    1500.00 ms (1)
        0 CTF compensation matrices available
        nave = 234 - aspect 

<ipython-input-4-ce1d908fed66>:44: RuntimeWarning: This filename (/content/drive/MyDrive/dataset/adhderps/21_train_erp.fif) does not conform to MNE naming conventions. All evoked files should end with -ave.fif, -ave.fif.gz, _ave.fif or _ave.fif.gz
  evoked = mne.read_evokeds(os.path.join(erp_folder, fname), condition=0)
<ipython-input-4-ce1d908fed66>:44: RuntimeWarning: This filename (/content/drive/MyDrive/dataset/adhderps/8_train_erp.fif) does not conform to MNE naming conventions. All evoked files should end with -ave.fif, -ave.fif.gz, _ave.fif or _ave.fif.gz
  evoked = mne.read_evokeds(os.path.join(erp_folder, fname), condition=0)
<ipython-input-4-ce1d908fed66>:44: RuntimeWarning: This filename (/content/drive/MyDrive/dataset/adhderps/43_train_erp.fif) does not conform to MNE naming conventions. All evoked files should end with -ave.fif, -ave.fif.gz, _ave.fif or _ave.fif.gz
  evoked = mne.read_evokeds(os.path.join(erp_folder, fname), condition=0)
<ipython-input-4-ce1d908fed66>:44: 

--- N1 Component ---
Channel: C4
Time Window: 120.0 - 180.0 ms
Peak Latency: 148.44 ms
Peak Amplitude: -28699472.43 µV

--- P3 Component ---
Channel: P7
Time Window: 300.0 - 500.0 ms
Peak Latency: 347.66 ms
Peak Amplitude: 61297092.44 µV


Processing: 39_train_erp.fif
Reading /content/drive/MyDrive/dataset/adhderps/39_train_erp.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    1500.00 ms (1)
        0 CTF compensation matrices available
        nave = 269 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
No baseline correction applied
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
--- P1 Component ---
Channel: P7
Time Window: 80.0 - 120.0 ms
Peak Latency: 117.19 ms
Peak Amplitude: 87229133.61 µV

--- N1 Component ---
Channel: AF4
Time Window: 120.0 - 180.0 ms
Peak Latency: 175.78 ms
Peak Amplitude: -44851612.09 µV

--- P3 Component ---
Channel: C3
Time Window: 300

<ipython-input-4-ce1d908fed66>:44: RuntimeWarning: This filename (/content/drive/MyDrive/dataset/adhderps/39_train_erp.fif) does not conform to MNE naming conventions. All evoked files should end with -ave.fif, -ave.fif.gz, _ave.fif or _ave.fif.gz
  evoked = mne.read_evokeds(os.path.join(erp_folder, fname), condition=0)
